In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models

# ✅ Device configuration (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ✅ Data transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),
                         (0.229, 0.224, 0.225))
])

# ✅ Datasets & DataLoader
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

# ✅ Model setup
model = models.resnet18(weights=None)
model.fc = nn.Linear(model.fc.in_features, 10)
model = model.to(device)

# ✅ Loss & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# ✅ Training loop with progress display
num_epochs = 
total_steps = len(train_loader)
total_images = len(train_dataset)

for epoch in range(num_epochs):
    running_loss = 0.0
    for batch_idx, (imgs, labels) in enumerate(train_loader):
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Print progress
        processed = (batch_idx + 1) * imgs.size(0)
        print(f"Epoch [{epoch+1}/{num_epochs}] | "
              f"Step [{batch_idx+1}/{total_steps}] | "
              f"Images: {processed}/{total_images} | "
              f"Loss: {loss.item():.4f}", end="\r")

    print(f"\n✅ Epoch [{epoch+1}/{num_epochs}] completed. "
          f"Average Loss: {running_loss / total_steps:.4f}\n")

print("🎉 Training complete!")

In [ ]:
import matplotlib.pyplot as plt
import torch

# Assuming 'imgs' and 'outputs' are from your last training batch
# Get predicted class
_, predicted = torch.max(outputs, 1)

# ✅ Denormalize images (since you used ImageNet normalization)
mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)

# Move tensors to CPU for plotting
imgs_cpu = imgs.detach().cpu()
predicted_cpu = predicted.detach().cpu()

# ✅ Denormalize
images_denorm = imgs_cpu * std + mean
images_denorm = torch.clamp(images_denorm, 0, 1)  # keep pixel range valid

# ✅ Plot 15 images with predictions
fig, axes = plt.subplots(1, 15, figsize=(15, 3))
for i in range(15):
    ax = axes[i]
    ax.imshow(images_denorm[i].permute(1, 2, 0))  # C×H×W → H×W×C
    ax.set_title(f"Pred: {predicted_cpu[i].item()}", fontsize=8)
    ax.axis('off')

plt.tight_layout()
plt.show()